In [2]:
import word2vec
import pandas as pd
import jieba
import jieba.analyse
from opencc import OpenCC

cc = OpenCC('s2t')

D:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
fd = open('D://NLP/wiki_zh_2019/wiki_zh/AA/wiki_00', "r", encoding="utf-8")

id_list    = []
url_list   = []
title_list = []
text_list  = []

lines = fd.readlines()
for line in lines:
    line = line.strip('')

    id_value    = line.split(', ')[0].strip('{').split(' ')[1].strip('"')
    url_value   = line.split(', ')[1].split(' ')[1].strip('"')
    title_value = cc.convert(str(line.split(', ')[2].split(' ')[1].strip('"')))
    text_value  = cc.convert(str(line.split(', ')[3].split(': "')[1].strip('"\'}\n')))
    
    id_list.append(id_value)
    url_list.append(url_value)
    title_list.append(title_value)
    text_list.append(text_value)

fd.close()

In [4]:
def remove_stop_words(filename, word_list):
    with open(filename, 'r', encoding="utf-8") as f:
        stop_words = f.read().split('\n')
        
    stop_words.append('\n')
    stop_words.append('\n\n')
    stop_words.append('\n\n\n')
    stop_words.append('\n\n\n\n')
    stop_words.append('\\')
    stop_words.append('/')
    stop_words.append(' ')
    stop_words.append('n')
    stop_words.append('{')
    stop_words.append('}')
    stop_words.append('－')
    
    new_list = []
    
    for word in word_list:
        if word not in stop_words:
            new_list.append(word)
    
    return new_list

In [14]:
cut_text_list = []
#print(text_list[0])

for text in text_list:
    text = jieba.lcut(text)
    text = remove_stop_words('stop.txt', text)
    
    cut_text_list.append(text)


wiki_dict = {'id': id_list, 'url': url_list, 'title': title_list, 'text': cut_text_list} 
wiki_df = pd.DataFrame(wiki_dict)

print(wiki_df.iloc[0])

id                                                      13
url                 https://zh.wikipedia.org/wiki?curid=13
title                                                   數學
text     [數學, 數學, 利用, 符號, 語言, 研究, 數量, 結構, 變化, 空間, 概念, 一...
Name: 0, dtype: object


In [7]:
import numpy
numpy.__version__
from gensim.models.word2vec import Word2Vec

In [8]:
test_model = Word2Vec(cut_text_list)

In [11]:
similar_word = test_model.wv.most_similar('數學', topn=10)
print(similar_word)

[('領域', 0.9992307424545288), ('科學', 0.9990180730819702), ('研究', 0.9989759922027588), ('應用', 0.9988747835159302), ('物理', 0.9988300800323486), ('方法', 0.9987704753875732), ('學科', 0.9987117052078247), ('生物', 0.9986189007759094), ('理論', 0.9985644221305847), ('心理', 0.9984905123710632)]


In [28]:
import time


file_prefix = ['AA/', 'AB/', 'AC/', 'AD/', 'AE/', 'AF/', 'AG/', 'AH/', 'AI/', 'AJ/']
#file_prefix = ['AK/', 'AL/', 'AM/']


v2_id_list    = []
v2_url_list   = []
v2_title_list = []
v2_text_list  = []


start_t = time.time()

for prefix in file_prefix:
    print(prefix)
    
    #for i in range(0, 5):
    for i in range(0, 100):
        fn = 'D://NLP/wiki_zh_2019/wiki_zh/' + prefix + 'wiki_' + str("{:02}".format(i))
        #print(fn)
        
        fd = open(fn, "r", encoding="utf-8")
        
        lines = fd.readlines()
        for line in lines:
            line = line.strip('')

            id_value    = line.split(', ')[0].strip('{').split(' ')[1].strip('"')
            url_value   = line.split(', ')[1].split(' ')[1].strip('"')
            title_value = cc.convert(str(line.split(', ')[2].split(' ')[1].strip('"')))
            text_value  = cc.convert(str(line.split(', ')[3].split(': "')[1].strip('"\'}\n')))

            v2_id_list.append(id_value)
            v2_url_list.append(url_value)
            v2_title_list.append(title_value)
            v2_text_list.append(text_value)

        fd.close()

        
v2_cut_text_list = []
#print(text_list[0])

for text in v2_text_list:
    text = jieba.lcut(text)
    text = remove_stop_words('stop.txt', text)
    
    v2_cut_text_list.append(text)



v2_test_model = Word2Vec(v2_cut_text_list)

end_t = time.time()

similar_word = v2_test_model.wv.most_similar('數學', topn=20)

print("total time: ", end_t - start_t, " s\n")
print(similar_word)

AA/
AB/
total time:  63.54530382156372  s

[('邏輯', 0.9579820036888123), ('分析', 0.9552180767059326), ('理論', 0.9497487545013428), ('語義學', 0.9445591568946838), ('物理', 0.9444211721420288), ('語言學', 0.9374140501022339), ('性質', 0.9371500611305237), ('數學理論', 0.9365296363830566), ('模型', 0.9364597201347351), ('數論', 0.9348641037940979), ('數學家', 0.933047890663147), ('歸納', 0.9316354393959045), ('機科學', 0.9295642971992493), ('邏輯學', 0.9279452562332153), ('學中', 0.9254655838012695), ('力學', 0.9242505431175232), ('形而上', 0.9230303764343262), ('生理', 0.9217222332954407), ('知識', 0.9197220802307129), ('純數學', 0.9188389182090759)]
